In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
import glob
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from scipy.stats import friedmanchisquare
from scikit_posthocs import posthoc_nemenyi_friedman as nemenyi
from quality.create_quality_df import create_pred_df, calculate_measures
import sys
sys.path.append("/home/dobraczka/Downloads/git/CSVtoLaTeXTable/colorizelatextables")
from colorize_table import to_colorized_latex, add_avg_rank

In [3]:
def get_new_values():
    datasets = ["D_W_15K_V1",
            "D_W_15K_V2",
            "D_Y_15K_V1",
            "D_Y_15K_V2",
            "EN_DE_15K_V1",
            "EN_DE_15K_V2",
            "EN_FR_15K_V1",
            "EN_FR_15K_V2",
            "D_W_100K_V1",
            "D_W_100K_V2",
            "D_Y_100K_V1",
            "D_Y_100K_V2",
            "EN_DE_100K_V1",
            "EN_DE_100K_V2",
            "EN_FR_100K_V1",
            "EN_FR_100K_V2",
            "abt-buy",
            "amazon-google",
            "dblp-scholar",
            "dblp-acm",
            "imdb-tmdb",
            "tmdb-tvdb",
            "imdb-tvdb"]
    app = ["boot_ea","rdgcn","multi_ke"]
    approaches = ["BootEA","RDGCN","MultiKE"]
    frames = []
    for ds_name in datasets:
        for fold in range(1,6):
            for embed_approach,embed in zip(approaches,app):
                file_path = glob.glob(f"/home/dobraczka/Downloads/git/er-embedding-benchmark/data/results*/output/results/{ds_name}-721_5fold-{fold}/{embed_approach}/datasets/*/*{embed}.csv")[0]
                df = pd.read_csv(file_path)
                df["embed_model"] = embed_approach
                df["fold"] = fold
                df["dataset"] = ds_name
                frames.append(df)
    return pd.concat(frames)

In [8]:
df = get_new_values()
df["model_name"] = df["model_name"].map({"ada boost":"Ada","decision tree":"DT","gaussian naive bayes":"Bayes","random forest 500":"RF","svc":"svc","MLP":"MLP","logistic regression":"lr"})
rich_graph_datasets ={
        "D_W_15K_V1",
    "D_W_15K_V2",
    "D_Y_15K_V1",
    "D_Y_15K_V2",
    "EN_DE_15K_V1",
    "EN_DE_15K_V2",
    "EN_FR_15K_V1",
    "EN_FR_15K_V2",
    "D_W_100K_V1",
    "D_W_100K_V2",
    "D_Y_100K_V1",
    "D_Y_100K_V2",
    "EN_DE_100K_V1",
    "EN_DE_100K_V2",
    "EN_FR_100K_V1",
    "EN_FR_100K_V2",
}
only_sim = df[df["vector_name"]=="OnlySim"].groupby(["model_name","dataset","fold"]).mean().reset_index()
only_sim["vector_name"] = "OnlySim"
only_sim["embed_model"] = ""
df = df[df["vector_name"]!="OnlySim"]
df = pd.concat([df,only_sim])
allowed_models={"MLP","RF"}
allowed_vectors={"OnlySim","OnlyEmb","SimAndEmb"}
df = df[df["model_name"].isin(allowed_models)]
df = df[df["vector_name"].isin(allowed_vectors)]
df.rename(columns= {"test_f1":"fm","test_precision":"prec","test_recall":"rec"},inplace=True)
df = df[["model_name","vector_name","dataset","embed_model","fm","prec","rec","fold"]]
rich_df = df[df["dataset"].isin(rich_graph_datasets)]
rich_df["ds_size"] = rich_df["dataset"].map(lambda x: x.split("_")[2])
rich_df["dataset"] = rich_df["dataset"].map(lambda x: x.replace("15K","").replace("100K","").replace("_","-").replace("--V1","(V1)").replace("--V2","(V2)"))
r_small = rich_df[rich_df["ds_size"]=="15K"]
r_large = rich_df[rich_df["ds_size"]=="100K"]
shallow_df = df[~df["dataset"].isin(rich_graph_datasets)]

/home/dobraczka/.local/share/virtualenvs/er-embedding-benchmark-_gu3Nn8a/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/dobraczka/.local/share/virtualenvs/er-embedding-benchmark-_gu3Nn8a/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dobraczka/.local/share/virtualenvs/er-embedding-benchmark-_gu3Nn8a/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [11]:
shallow_df.pivot_table(index=["dataset"],columns=["vector_name","embed_model","model_name"],values=["fm"])

fm                                                    \
vector_name     OnlyEmb                                                     
embed_model      BootEA             MultiKE               RDGCN             
model_name          MLP        RF       MLP        RF       MLP        RF   
dataset                                                                     
abt-buy        0.623165  0.647509  0.383427  0.655464  0.650222  0.661023   
amazon-google  0.630617  0.645993  0.570928  0.644658  0.637552  0.664668   
dblp-acm       0.578911  0.614159  0.616990  0.688298  0.558755  0.597792   
dblp-scholar   0.561579  0.587793  0.537368  0.575676  0.547317  0.571210   
imdb-tmdb      0.874261  0.859323  0.910813  0.912608  0.874125  0.873433   
imdb-tvdb      0.820741  0.785821  0.872656  0.843994  0.807487  0.791721   
tmdb-tvdb      0.874259  0.844239  0.870932  0.877100  0.856621  0.830947   

                                                                           \
vector_name     OnlySim           SimAndEmb                                 
embed_model                          BootEA             MultiKE             
model_name          MLP        RF       MLP        RF       MLP        RF   
dataset                                                                     
abt-buy        0.977471  0.976577  0.960932  0.972812  0.971963  0.972309   
amazon-google  0.985293  0.982373  0.953017  0.977549  0.975781  0.977777   
dblp-acm       0.996953  0.996786  0.994779  0.996844  0.996779  0.997102   
dblp-scholar   0.997190  0.997584  0.993351  0.997378  0.993973  0.997271   
imdb-tmdb      0.992607  0.994724  0.994693  0.994530  0.996726  0.995847   
imdb-tvdb      0.967605  0.995620  0.995750  0.995357  0.996793  0.995876   
tmdb-tvdb      0.890645  0.994800  0.995977  0.994670  0.995846  0.995967   

                                   
vector_name                        
embed_model       RDGCN            
model_name          MLP        RF  
dataset                            
abt-buy        0.964152  0.973487  
amazon-google  0.964894  0.975479  
dblp-acm       0.994837  0.996843  
dblp-scholar   0.994882  0.996680  
imdb-tmdb      0.994680  0.995193  
imdb-tvdb      0.994264  0.996008  
tmdb-tvdb      0.994412  0.994802

In [12]:
rich_df.pivot_table(index=["dataset"],columns=["vector_name","embed_model","model_name"],values=["fm"])

fm                                                    \
vector_name   OnlyEmb                                                     
embed_model    BootEA             MultiKE               RDGCN             
model_name        MLP        RF       MLP        RF       MLP        RF   
dataset                                                                   
D-W(V1)      0.816256  0.748877  0.851670  0.871043  0.681626  0.631805   
D-W(V2)      0.948820  0.881697  0.928132  0.949205  0.709010  0.702312   
D-Y(V1)      0.898044  0.844228  0.950435  0.947300  0.955021  0.960330   
D-Y(V2)      0.976846  0.922780  0.963182  0.974727  0.933887  0.963927   
EN-DE(V1)    0.895755  0.828558  0.956262  0.948142  0.904819  0.922859   
EN-DE(V2)    0.940084  0.882555  0.965390  0.963686  0.856461  0.901409   
EN-FR(V1)    0.835895  0.763722  0.950219  0.942746  0.886226  0.914378   
EN-FR(V2)    0.933476  0.875794  0.968154  0.972646  0.831850  0.876984   

                                                                         \
vector_name   OnlySim           SimAndEmb                                 
embed_model                        BootEA             MultiKE             
model_name        MLP        RF       MLP        RF       MLP        RF   
dataset                                                                   
D-W(V1)      0.711585  0.733301  0.839998  0.846187  0.869747  0.846639   
D-W(V2)      0.752159  0.792059  0.958731  0.938432  0.945965  0.926776   
D-Y(V1)      0.962552  0.962345  0.960414  0.969970  0.991765  0.991618   
D-Y(V2)      0.972772  0.973011  0.993437  0.985995  0.995921  0.994563   
EN-DE(V1)    0.873554  0.899289  0.942504  0.925642  0.980758  0.975032   
EN-DE(V2)    0.863822  0.904606  0.965188  0.947534  0.984925  0.981296   
EN-FR(V1)    0.839614  0.857310  0.906747  0.902730  0.970606  0.964739   
EN-FR(V2)    0.824784  0.872857  0.951760  0.935976  0.973805  0.980680   

                                 
vector_name                      
embed_model     RDGCN            
model_name        MLP        RF  
dataset                          
D-W(V1)      0.757922  0.780184  
D-W(V2)      0.824726  0.821872  
D-Y(V1)      0.983570  0.991388  
D-Y(V2)      0.983476  0.991173  
EN-DE(V1)    0.954059  0.957149  
EN-DE(V2)    0.936448  0.948932  
EN-FR(V1)    0.942135  0.945932  
EN-FR(V2)    0.918506  0.941799